In [ ]:
import pyvista as pv
from pyvistaqt import BackgroundPlotter
import matplotlib.pyplot as plt

import glob
import cv2
import numpy as np
from scipy.ndimage import uniform_filter

from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)

In [ ]:
ret_path = '3d_dataset/retardance3D/*'
azimuth_path = '3d_dataset/azimuth/*'
theta_path = '3d_dataset/theta/*'

ret_files = sorted(glob.glob(ret_path))
azimuth_files = sorted(glob.glob(azimuth_path))
theta_files = sorted(glob.glob(theta_path))

In [ ]:
N_z = 96

ret = []
azimuth = []
theta = []
for i in range(N_z):
    ret.append(cv2.imread(ret_files[i], -1).astype('float32'))
    azimuth.append(cv2.imread(azimuth_files[i], -1).astype('float32'))
    theta.append(cv2.imread(theta_files[i], -1).astype('float32'))

ret = np.transpose(np.array(ret),(1,2,0))/10000
azimuth = np.transpose(np.array(azimuth),(1,2,0))/18000*np.pi
theta = np.transpose(np.array(theta),(1,2,0))/18000*np.pi

In [ ]:
anisotropy=0.2*np.abs(ret)
linelength = 20

f_1c = anisotropy*linelength*(np.sin(theta)**2)*np.cos(2*azimuth)
f_1s = anisotropy*linelength*(np.sin(theta)**2)*np.sin(2*azimuth)
f_2c = anisotropy*linelength*np.sin(2*theta)*np.cos(azimuth)
f_2s = anisotropy*linelength*np.sin(2*theta)*np.sin(azimuth)

f_1c_denoised = denoise_tv_chambolle(f_1c, weight=2)
f_1s_denoised = denoise_tv_chambolle(f_1s, weight=2)
f_2c_denoised = denoise_tv_chambolle(f_2c, weight=2)
f_2s_denoised = denoise_tv_chambolle(f_2s, weight=2)

# Averaging
f_1c_smooth = uniform_filter(f_1c_denoised)
f_1s_smooth = uniform_filter(f_1s_denoised)
f_2c_smooth = uniform_filter(f_2c_denoised)
f_2s_smooth = uniform_filter(f_2s_denoised)

In [ ]:
reg_ret_ap = 0.0001
azimuth_n = (np.arctan2(f_1s_smooth, f_1c_smooth)/2)%np.pi
del_f_sin_square_n = f_1s_smooth*np.sin(2*azimuth_n) + f_1c_smooth*np.cos(2*azimuth_n)
del_f_sin2theta_n = f_2s_smooth*np.sin(azimuth_n) + f_2c_smooth*np.cos(azimuth_n)
theta_n = np.arctan2(2*del_f_sin_square_n, del_f_sin2theta_n)
retardance_ap_n = del_f_sin_square_n * np.sin(theta_n)**2 / (np.sin(theta_n)**4 + reg_ret_ap)

In [ ]:
USmooth = retardance_ap_n*np.cos(azimuth_n)*np.sin(theta_n)
VSmooth = retardance_ap_n*np.sin(azimuth_n)*np.sin(theta_n)
WSmooth = retardance_ap_n*np.cos(theta_n)

nY, nX, nZ = USmooth.shape
y, x, z = np.mgrid[0:nY,0:nX,0:nZ]
spacing=5
spacing_z=5

# Plot sparsely sampled vector lines
Plotting_X = x[::-spacing, ::spacing, ::spacing_z]
Plotting_Y = y[::-spacing, ::spacing, ::spacing_z]
Plotting_Z = z[::-spacing, ::spacing, ::spacing_z]
Plotting_U = USmooth[::-spacing, ::spacing, ::spacing_z]
Plotting_V = VSmooth[::-spacing, ::spacing, ::spacing_z]
Plotting_W = WSmooth[::-spacing, ::spacing, ::spacing_z]

points = np.empty((Plotting_X.size, 3))
points[:, 0] = Plotting_X.ravel('F')
points[:, 1] = Plotting_Y.ravel('F')
points[:, 2] = Plotting_Z.ravel('F')

directions = np.empty((Plotting_X.size, 3))
directions[:, 0] = Plotting_U.ravel('F')
directions[:, 1] = Plotting_V.ravel('F')
directions[:, 2] = Plotting_W.ravel('F')

In [ ]:
p = BackgroundPlotter()
p.clear()  # clears all actors
p.add_arrows(points, directions*2, 1)

In [ ]:
p.clear()  # clears all actors

In [ ]:
ret_path = '3d_dataset_2/retardance3D/*'
azimuth_path = '3d_dataset_2/azimuth/*'
theta_path = '3d_dataset_2/theta/*'

ret_files = sorted(glob.glob(ret_path))
azimuth_files = sorted(glob.glob(azimuth_path))
theta_files = sorted(glob.glob(theta_path))

In [ ]:
ret = []
azimuth = []
theta = []
for i in range(N_z):
    ret.append(cv2.imread(ret_files[i], -1).astype('float32'))
    azimuth.append(cv2.imread(azimuth_files[i], -1).astype('float32'))
    theta.append(cv2.imread(theta_files[i], -1).astype('float32'))

ret = np.transpose(np.array(ret),(1,2,0))/10000
azimuth = np.transpose(np.array(azimuth),(1,2,0))/18000*np.pi
theta = np.transpose(np.array(theta),(1,2,0))/18000*np.pi

In [ ]:
anisotropy=0.2*np.abs(ret)
linelength = 20

f_1c = anisotropy*linelength*(np.sin(theta)**2)*np.cos(2*azimuth)
f_1s = anisotropy*linelength*(np.sin(theta)**2)*np.sin(2*azimuth)
f_2c = anisotropy*linelength*np.sin(2*theta)*np.cos(azimuth)
f_2s = anisotropy*linelength*np.sin(2*theta)*np.sin(azimuth)

f_1c_denoised = denoise_tv_chambolle(f_1c, weight=2)
f_1s_denoised = denoise_tv_chambolle(f_1s, weight=2)
f_2c_denoised = denoise_tv_chambolle(f_2c, weight=2)
f_2s_denoised = denoise_tv_chambolle(f_2s, weight=2)

# Averaging
f_1c_smooth = uniform_filter(f_1c_denoised, (40, 40, 10))
f_1s_smooth = uniform_filter(f_1s_denoised, (40, 40, 10))
f_2c_smooth = uniform_filter(f_2c_denoised, (40, 40, 10))
f_2s_smooth = uniform_filter(f_2s_denoised, (40, 40, 10))

# Denoising

reg_ret_ap = 0.0001
# azimuth_n = (np.arctan2(f_1s_smooth, f_1c_smooth)/2)%np.pi
# del_f_sin_square_n = f_1s_smooth*np.sin(2*azimuth_n) + f_1c_smooth*np.cos(2*azimuth_n)
# del_f_sin2theta_n = f_2s_smooth*np.sin(azimuth_n) + f_2c_smooth*np.cos(azimuth_n)
# theta_n = np.arctan2(2*del_f_sin_square_n, del_f_sin2theta_n)
# retardance_ap_n = del_f_sin_square_n * np.sin(theta_n)**2 / (np.sin(theta_n)**4 + reg_ret_ap)

# Negative Retardance
azimuth_n = (np.arctan2(-f_1s_smooth, -f_1c_smooth)/2)%np.pi
del_f_sin_square_n = f_1s_smooth*np.sin(2*azimuth_n) + f_1c_smooth*np.cos(2*azimuth_n)
del_f_sin2theta_n = f_2s_smooth*np.sin(azimuth_n) + f_2c_smooth*np.cos(azimuth_n)
theta_n = np.arctan2(-2*del_f_sin_square_n, -del_f_sin2theta_n)
retardance_ap_n = del_f_sin_square_n * np.sin(theta_n)**2 / (np.sin(theta_n)**4 + reg_ret_ap)

In [ ]:
retardance_ap_n = np.load('retardance_ap_n.npy')
azimuth_n = np.load('azimuth_n.npy')
theta_n = np.load('theta_n.npy')

In [ ]:
USmooth = retardance_ap_n*np.cos(azimuth_n)*np.sin(theta_n)
VSmooth = retardance_ap_n*np.sin(azimuth_n)*np.sin(theta_n)
WSmooth = retardance_ap_n*np.cos(theta_n)

# Make the grid
nY, nX, nZ = USmooth.shape
# Make a grid
y, x, z = np.mgrid[0:nY,0:nX,0:nZ]
spacing=7
spacing_z=2

# Plot sparsely sampled vector lines
Plotting_X = x[::-spacing, ::spacing, ::spacing_z]
Plotting_Y = y[::-spacing, ::spacing, ::spacing_z]
Plotting_Z = z[::-spacing, ::spacing, ::spacing_z]
Plotting_U = USmooth[::-spacing, ::spacing, ::spacing_z]
Plotting_V = VSmooth[::-spacing, ::spacing, ::spacing_z]
Plotting_W = WSmooth[::-spacing, ::spacing, ::spacing_z]

points = np.empty((Plotting_X.size, 3))
points[:, 0] = Plotting_X.ravel('F')
points[:, 1] = Plotting_Y.ravel('F')
points[:, 2] = Plotting_Z.ravel('F')

directions = np.empty((Plotting_X.size, 3))
directions[:, 0] = Plotting_U.ravel('F')
directions[:, 1] = Plotting_V.ravel('F')
directions[:, 2] = Plotting_W.ravel('F')

In [ ]:
p = BackgroundPlotter()
p.clear()
p.add_arrows(points, directions, 2)

In [ ]:
p.clear()